In [1]:
pip install opencv-python numpy scikit-image

Note: you may need to restart the kernel to use updated packages.


In [5]:
!dir

 Volume in drive C is Windows-SSD
 Volume Serial Number is 3893-BB6D

 Directory of C:\Users\korra\Downloads\YOLO_fine_tune

27-05-2025  13:12    <DIR>          .
27-05-2025  12:28    <DIR>          ..
27-05-2025  12:33    <DIR>          .ipynb_checkpoints
27-05-2025  13:13    <DIR>          Clean_outputs
27-05-2025  10:18    <DIR>          data_annotated
27-05-2025  12:35            34,202 dehazing.ipynb
27-05-2025  12:13           343,533 fine_tuning_YOLO.ipynb
27-05-2025  07:33    <DIR>          Foggy_Cityscapes
27-05-2025  13:12            17,393 preprocessing_images.ipynb
27-05-2025  10:49    <DIR>          runs
27-05-2025  10:49        22,588,772 yolov8s.pt
               4 File(s)     22,983,900 bytes
               7 Dir(s)  72,133,443,584 bytes free


In [ ]:
import cv2
import numpy as np
import os
import csv
import math
from skimage.metrics import mean_squared_error 
from skimage import io, img_as_float 

In [ ]:
def calculate_entropy(image_gray):
    if image_gray is None:
        return None
    hist = cv2.calcHist([image_gray], [0], None, [256], [0, 256])
    hist_norm = hist.ravel() / hist.sum()
    entropy = -np.sum(hist_norm * np.log2(hist_norm + np.finfo(float).eps))
    return entropy

In [ ]:
def dark_channel(im, patch_size):
    b, g, r = cv2.split(im)
    dc = np.minimum(np.minimum(r, g), b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (patch_size, patch_size))
    dark = cv2.erode(dc, kernel)
    return dark

In [ ]:
def atmospheric_light(im, dark_channel_img, p=0.001):
    im_flat = im.reshape(-1, 3)
    dark_flat = dark_channel_img.flatten()
    num_pixels = dark_flat.shape[0]
    num_search_pixels = int(max(math.floor(num_pixels * p), 1))
    indices = np.argsort(dark_flat)[-num_search_pixels:]
    atm_sum = np.zeros(3)
    for idx in indices:
        atm_sum += im_flat[idx]
    A = atm_sum / num_search_pixels
    return A.astype(np.float32)

In [ ]:
def dehaze_dcp(image, patch_size=15, omega=0.95, t0=0.1, refine_radius=40):
    if image is None:
        print("Error: DCP received a None image.")
        return None
    
    original_dtype = image.dtype
    if not np.issubdtype(original_dtype, np.uint8):
        print(f"Warning: DCP input image is not uint8 ({original_dtype}), converting. This might affect results.")
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    img_float_01 = image.astype(np.float32) / 255.0 # DCP works well with [0,1] range

    dc = dark_channel(image, patch_size) # Dark channel on original 0-255 image
    A = atmospheric_light(image, dc, p=0.001) # Atmospheric light from original 0-255 image

    A_01 = A / 255.0
    im_norm_by_A = np.empty_like(img_float_01)
    for i in range(3):
        if A_01[i] > 1e-5:
            im_norm_by_A[:,:,i] = img_float_01[:,:,i] / A_01[i]
        else:
            im_norm_by_A[:,:,i] = img_float_01[:,:,i] # Avoid division by zero

    transmission_raw = 1 - omega * (dark_channel( (im_norm_by_A * 255).astype(np.uint8) , patch_size) / 255.0) # Dark channel of normalized image
    transmission = np.maximum(transmission_raw, t0)

    # Refine transmission
    try:
        import cv2.ximgproc
        gray_guide = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
        refined_t = cv2.ximgproc.guidedFilter(guide=gray_guide,
                                              src=transmission.astype(np.float32),
                                              radius=refine_radius,
                                              eps=0.001) # Regularization
    except (ImportError, AttributeError, cv2.error) as e:
        if transmission.ndim == 3 and transmission.shape[2] == 1:
            t_for_blur = transmission[:,:,0].astype(np.float32)
        elif transmission.ndim == 2:
            t_for_blur = transmission.astype(np.float32)
        else: # Fallback if transmission is unexpectedly multi-channel > 1
            t_for_blur = cv2.cvtColor(transmission.astype(np.float32), cv2.COLOR_BGR2GRAY)

        refined_t_blur = cv2.medianBlur(t_for_blur, 2 * (refine_radius // 2) + 1) # Kernel size must be odd
        if refined_t_blur.ndim == 2: 
             refined_t = refined_t_blur
        else:
             refined_t = refined_t_blur


  
    if refined_t.ndim == 2:
        t_broadcast = refined_t[:, :, np.newaxis] # Make it (H, W, 1) for broadcasting
    elif refined_t.shape[2] == 1:
        t_broadcast = refined_t
    else: # If refined_t is 3-channel from some other refinement, use as is
        t_broadcast = refined_t


    J = (image.astype(np.float32) - A) / np.maximum(t_broadcast, t0) + A
    J = np.clip(J, 0, 255).astype(np.uint8)
    return J


In [ ]:
def apply_clahe(image_bgr, clip_limit=2.0, tile_grid_size=(8,8)):
    if image_bgr is None:
        print("Error: CLAHE received a None image.")
        return None
    lab = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe_obj = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    cl = clahe_obj.apply(l)
    limg = cv2.merge((cl, a, b))
    final_clahe = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final_clahe

In [9]:
if __name__ == "__main__":
    # --- Configuration ---
    input_image_dir = 'Foggy_Cityscapes/Dense_Fog'  # <--- CHANGE THIS TO YOUR INPUT FOLDER
    output_dcp_dir = 'Clean_outputs/DCP'
    output_clahe_dir = 'Clean_outputs/CLAHE'
    output_csv_file = 'Clean_outputs/metrics.csv'

    dcp_patch_size = 15
    dcp_omega = 0.90      # Might need to be slightly lower for aggressive dehazing
    dcp_t0 = 0.1
    dcp_refine_radius = 50 # Increased radius for smoother transmission

    clahe_clip_limit = 2.0
    clahe_tile_grid = (8, 8)


    # Create output directories if they don't exist
    os.makedirs(output_dcp_dir, exist_ok=True)
    os.makedirs(output_clahe_dir, exist_ok=True)
    os.makedirs(os.path.dirname(output_csv_file), exist_ok=True)


    # Prepare CSV file
    csv_header = [
        'filename',
        'original_entropy',
        'dcp_processed', 'dcp_entropy',
        'clahe_processed', 'clahe_entropy'
    ]
    
    # Check if input directory exists
    if not os.path.isdir(input_image_dir):
        print(f"Error: Input directory '{input_image_dir}' does not exist. Please create it and add your images.")
        exit()

    image_files = [f for f in os.listdir(input_image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    
    if not image_files:
        print(f"No image files found in '{input_image_dir}'.")
        exit()
        
    print(f"Found {len(image_files)} images to process in '{input_image_dir}'.")

    with open(output_csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_header)

        for i, filename in enumerate(image_files):
            print(f"\nProcessing image {i+1}/{len(image_files)}: {filename}")
            input_path = os.path.join(input_image_dir, filename)
            
            original_image = cv2.imread(input_path)
            if original_image is None:
                print(f"  Error: Could not read original image {filename}. Skipping.")
                writer.writerow([filename, 'N/A', False, 'N/A', False, 'N/A'])
                continue

            original_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
            original_entropy = calculate_entropy(original_gray)

            row_data = {
                'filename': filename,
                'original_entropy': f"{original_entropy:.4f}" if original_entropy is not None else 'N/A',
                'dcp_processed': False, 'dcp_entropy': 'N/A',
                'clahe_processed': False, 'clahe_entropy': 'N/A'
            }

            # 1. Dark Channel Prior Dehazing
            print(f"  Applying DCP...")
            try:
                dehazed_dcp_image = dehaze_dcp(original_image.copy(),
                                               patch_size=dcp_patch_size,
                                               omega=dcp_omega,
                                               t0=dcp_t0,
                                               refine_radius=dcp_refine_radius)
                if dehazed_dcp_image is not None:
                    output_path_dcp = os.path.join(output_dcp_dir, filename)
                    cv2.imwrite(output_path_dcp, dehazed_dcp_image)
                    dcp_gray = cv2.cvtColor(dehazed_dcp_image, cv2.COLOR_BGR2GRAY)
                    dcp_entropy = calculate_entropy(dcp_gray)
                    row_data['dcp_processed'] = True
                    row_data['dcp_entropy'] = f"{dcp_entropy:.4f}" if dcp_entropy is not None else 'N/A'
                    print(f"    DCP image saved to {output_path_dcp}")
                else:
                    print(f"    DCP processing failed for {filename}.")
            except Exception as e:
                print(f"    Exception during DCP processing for {filename}: {e}")


            # 2. CLAHE Enhancement
            print(f"  Applying CLAHE...")
            try:
                clahe_image = apply_clahe(original_image.copy(),
                                          clip_limit=clahe_clip_limit,
                                          tile_grid_size=clahe_tile_grid)
                if clahe_image is not None:
                    output_path_clahe = os.path.join(output_clahe_dir, filename)
                    cv2.imwrite(output_path_clahe, clahe_image)
                    clahe_gray = cv2.cvtColor(clahe_image, cv2.COLOR_BGR2GRAY)
                    clahe_entropy = calculate_entropy(clahe_gray)
                    row_data['clahe_processed'] = True
                    row_data['clahe_entropy'] = f"{clahe_entropy:.4f}" if clahe_entropy is not None else 'N/A'
                    print(f"    CLAHE image saved to {output_path_clahe}")
                else:
                    print(f"    CLAHE processing failed for {filename}.")
            except Exception as e:
                print(f"    Exception during CLAHE processing for {filename}: {e}")

            writer.writerow([
                row_data['filename'],
                row_data['original_entropy'],
                row_data['dcp_processed'], row_data['dcp_entropy'],
                row_data['clahe_processed'], row_data['clahe_entropy']
            ])

    print(f"\n\nBatch processing complete. Processed images saved in respective output folders.")
    print(f"Metrics saved to: {output_csv_file}")

Found 500 images to process in 'Foggy_Cityscapes/Dense_Fog'.

Processing image 1/500: 001.png
  Applying DCP...
    Exception during DCP processing for 001.png: cannot access local variable 'cv2' where it is not associated with a value
  Applying CLAHE...
    CLAHE image saved to Clean_outputs/CLAHE\001.png

Processing image 2/500: 002.png
  Applying DCP...
    Exception during DCP processing for 002.png: cannot access local variable 'cv2' where it is not associated with a value
  Applying CLAHE...
    CLAHE image saved to Clean_outputs/CLAHE\002.png

Processing image 3/500: 003.png
  Applying DCP...
    Exception during DCP processing for 003.png: cannot access local variable 'cv2' where it is not associated with a value
  Applying CLAHE...
    CLAHE image saved to Clean_outputs/CLAHE\003.png

Processing image 4/500: 004.png
  Applying DCP...
    Exception during DCP processing for 004.png: cannot access local variable 'cv2' where it is not associated with a value
  Applying CLAHE...


In [11]:
pip install onnx onnxruntime 


  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached pyreadline3-3.5.4-py3-none-any.whl.metadata (4.7 kB)
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
    --------------------------------------- 0.3/12.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.7 MB 2.1 MB/s eta 0:00:06
   - -------------------------------------- 0.5/12.7 MB 2.1 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/12.7 MB 931.2 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/12.7 MB 1.2 MB/s eta 0:00:10
   ---- ----------------------------------- 1.3/12.7 MB 1.2 MB/s eta 0:00:10
   ----- ---------------------------------- 1.8/12.7 MB 1.3 MB/s eta 0:00:09
   ------ -------------------------

In [ ]:
yolo export model=runs/detect/your_foggy_yolov8s_run1/weights/best.pt format=onnx imgsz=640 half=True